In [ ]:
from langchain_chroma import Chroma
import os
from pyprojroot import here

"""
The here() function from pyprojroot returns the root directory of the project, 
allowing you to build paths relative to that root. This is useful when you want your code to be portable across different systems without hardcoding the paths
"""


from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
load_dotenv()

In [2]:
# llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0,api_key="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")


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings


# from langchain.embeddings import HuggingFaceEmbeddings

# embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
# model_kwargs = {"device": "cuda"}

# embeddings = HuggingFaceEmbeddings(
#   model_name=embedding_model_name,
#   # model_kwargs=model_kwargs
# )

# Initialize the HuggingFace embeddings model
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2" , # Example model for generating embeddings
    model_kwargs = {"device": "cuda"}
)

# Example text to embed
texts = ["This is a sentence.", "Here's another text to be embedded.", "Embedding is useful for similarity search."]




In [ ]:
embedding_model.embed_query('how are you')

In [ ]:
embedding_model.embed_documents(texts=texts)

In [6]:
VECTORDB_DIR = "data/airline_policy_vectordb"
K=2


In [ ]:
vectordb = Chroma(
    
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=embedding_model
)
print("Number of vectors in vectordb:",
      vectordb._collection.count(), "\n\n")

In [8]:
message = "What is the cancelation rule for a flight ticket at swiss airline policy?"

In [9]:
docs = vectordb.similarity_search(message,k=2)

In [ ]:
docs

# prompt Preparation

In [11]:
message ="What is the cancelation rule for a flight ticket at swiss airline policy?"

In [12]:
question = "# User new question:\n" + message
retrieved_content = ""
for doc in docs:
    retrieved_content += f"{doc.page_content}\n\n"
prompt = f"# Content:\n{retrieved_content}\n\n{question}"

In [ ]:
print(prompt)

In [15]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0,api_key="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")
result = llm.invoke(prompt)

In [ ]:
result

# rag tool design using the lanchain

In [17]:
from langchain_core.tools import tool

In [18]:
from langchain_core.tools import tool

@tool
def lookup_swiss_airline_policy(query: str)->str:
    """Search within the Swiss Airline's company policies to check whether certain options are permitted. Input should be a search query."""
    vectordb = Chroma(
    collection_name="rag-chroma",
    persist_directory=str(here(VECTORDB_DIR)),
    embedding_function=embedding_model
    )
    docs = vectordb.similarity_search(query, k=K)
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
print(lookup_swiss_airline_policy.name)
print(lookup_swiss_airline_policy.args)
print(lookup_swiss_airline_policy.description)

In [ ]:
lookup_swiss_airline_policy.invoke("can i cancel tickets")

In [ ]:
message = "can I cancel my ticket? just answere yes or no in one word"

question = "# User new question:\n" + message

retrieved_content = lookup_swiss_airline_policy(message)

prompt = f"# Content:\n{retrieved_content}\n\n{question}"

In [ ]:
retrieved_content

In [22]:
llm = ChatGroq(model="mixtral-8x7b-32768", temperature=0,api_key="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")
result = llm.invoke(prompt)

In [ ]:
result.content